# 뗑컨 몸무게 예측 경진대회

## 패키지 import 및 데이터 전처리

In [33]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/train.csv')

In [5]:
df.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
0,0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.30515,-25.19017,5550
1,1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.63074,-24.34684,3800
2,2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.51951,-27.01854,4400
3,3,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,44.5,14.7,214,FEMALE,8.20106,-26.16524,4850
4,4,Gentoo penguin (Pygoscelis papua),Biscoe,No,49.6,16.0,225,MALE,8.38324,-26.84272,5700


|변수   |세부사항   |
|:-:|:-:|
|id | 샘플 아이디|
|Species| 펭귄의 종을 나타내는 문자열|
|Island | 샘플들이 수집된 Palmer Station 근처 섬 이름|
|Clutch Completion | 관찰된 펭귄 둥지의 알이 2개인 경우 Full Clutch이며 Yes로 표기|
|Culmen Length (mm) | 펭귄 옆모습 기준 부리의 가로 길이|
|Culmen Depth (mm) | 펭귄 옆모습 기준 부리의 세로 길이|
|Flipper Length (mm) |펭귄의 팔(날개) 길이|
|Sex | 펭귄의 성별|
|Delta 15 N (o/oo)  | 토양에 따라 변화하는 안정 동위원소 15N:14N의 비율|
|Delta 13 C (o/oo) | 먹이에 따라 변화하는 안정 동위원소 13C:12C의 비율|
|Body Mass (g)| 펭귄의 몸무게를 나타내는 숫자 (g)|

`-` 범주형 변수 4개, 양적 변수 6개이다

In [7]:
df.describe().round(2)

,id,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
count,114.00,114.00,114.00,114.00,111.00,111.00,114.00
mean,56.50,44.61,17.01,203.05,8.74,-25.72,4327.85
std,33.05,5.32,1.94,14.65,0.57,0.86,781.77
min,0.00,33.50,13.20,174.00,7.63,-27.02,2700.00
25%,28.25,40.33,15.22,190.00,8.27,-26.43,3675.00
50%,56.50,45.20,17.25,199.00,8.63,-25.96,4250.00
75%,84.75,49.08,18.60,216.00,9.26,-25.01,4850.00
max,113.00,55.10,21.10,231.00,10.03,-24.10,6300.00


In [16]:
df.shape ## 똉컨 114마리에 대한 10가지의 정보

(114, 11)

In [11]:
df.isnull().sum()

id                     0
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    3
Delta 15 N (o/oo)      3
Delta 13 C (o/oo)      3
Body Mass (g)          0
dtype: int64

`-` Sex, Delta 15 N, Delta 13 C 컬럼에 결측치가 3개씩 존재한다

In [90]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(df)

결측치가 있는 컬럼은: Sex입니다
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은: DeltaN입니다
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은: DeltaC입니다
해당 컬럼에 총 3개의 결측치가 존재합니다.


In [91]:
missing_col

[['Sex', dtype('O')],
 ['DeltaN', dtype('float64')],
 ['DeltaC', dtype('float64')]]

In [22]:
df.drop('id', axis = 1, inplace = True) ## id는 필요없으니 제거 

In [89]:
## 결측치가 있는 row들을 확인
df[df.isna().sum(axis = 1) > 0]

,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC,Body_Mass
6,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,42.0,20.2,190,NaN,9.13362,-25.09368,4250
8,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,34.1,18.1,193,NaN,NaN,NaN,3475
18,Adelie Penguin (Pygoscelis adeliae),Dream,No,39.8,19.1,184,MALE,NaN,NaN,4650
70,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.2,14.4,214,NaN,8.24253,-26.81540,4650
109,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.6,17.8,185,FEMALE,NaN,NaN,3700


`-` 결측치가 카테고리이면 행을 삭제하고 수치형이면 평균값을 넣어주자

In [93]:
# 결측치를 처리하는 함수 
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 카테고리형 feature가 결측치인 경우 해당 행들을 삭제
            temp = temp.dropna(subset=[col])
        elif dtype == int or dtype == float:
            # 수치형 feature가 결측치인 경우 평균값을 채움
            temp.loc[:,col] = temp[col].fillna(temp[col].mean())
    return temp

In [94]:
df = handle_na(df, missing_col)

In [95]:
# 결측치 처리가 잘 되었는지 확인
missing_col = check_missing_col(df) 

결측치가 존재하지 않습니다


In [28]:
df.rename(columns = {'Clutch Completion':'Full_Clutch', 'Culmen Length (mm)':'Culmen_Length', 
                     'Culmen Depth (mm)':'Culmen_Depth', 'Flipper Length (mm)':'Flipper_Length',
                     'Delta 15 N (o/oo)':'DeltaN', 'Delta 13 C (o/oo)':'DeltaC', 'Body Mass (g)':'Body_Mass'}, inplace = True) ## 열 이름 변경

`-` 분석의 편의를 위해 수치형 feature와 카테고리형 feature를 분리

In [115]:
# 전체의 데이터 타입을 확인
df.dtypes

Species            object
Island             object
Full_Clutch        object
Culmen_Length     float64
Culmen_Depth      float64
Flipper_Length      int64
Sex                object
DeltaN            float64
DeltaC            float64
Body_Mass           int64
dtype: object

In [118]:
numeric_feature = df.columns[(df.dtypes == np.int64)| (df.dtypes == float)]
categorical_feature = df.columns[df.dtypes == 'O']

print('수치형 데이터는 다음과 같습니다. \n', list(numeric_feature))
print('카테고리형 데이터는 다음과 같습니다. \n', list(categorical_feature))

수치형 데이터는 다음과 같습니다. 
 ['Culmen_Length', 'Culmen_Depth', 'Flipper_Length', 'DeltaN', 'DeltaC', 'Body_Mass']
카테고리형 데이터는 다음과 같습니다. 
 ['Species', 'Island', 'Full_Clutch', 'Sex']


## 모델링을 해보자

## 회귀 분석 (라벨 인코딩)

### 전처리

In [851]:
from sklearn.model_selection import train_test_split

In [865]:
dt = df.iloc[:, :-1]      ## target 제외한 데이터프레임
target = df['Body_Mass']  ## target

`-` 범주형 변수는 Species만 사용하자

In [866]:
dt2 = dt.drop(['Island', 'Full_Clutch', 'Sex'], axis = 1)

`-` 라벨인코딩을 해주자

In [867]:
dt2.head()

,Species,Culmen_Length,Culmen_Depth,Flipper_Length,DeltaN,DeltaC
0,Gentoo penguin (Pygoscelis papua),50.0,15.3,220,8.30515,-25.19017
1,Chinstrap penguin (Pygoscelis antarctica),49.5,19.0,200,9.63074,-24.34684
2,Gentoo penguin (Pygoscelis papua),45.1,14.4,210,8.51951,-27.01854
3,Gentoo penguin (Pygoscelis papua),44.5,14.7,214,8.20106,-26.16524
4,Gentoo penguin (Pygoscelis papua),49.6,16.0,225,8.38324,-26.84272


In [868]:
def labeling_penguin_name(name):
    if name == 'Gentoo penguin (Pygoscelis papua)':
        new = 2
    elif name == 'Adelie Penguin (Pygoscelis adeliae)':
        new = 1
    else:
        new = 0
    return new

In [869]:
dt3 = dt2.copy()

In [870]:
dt3['Species_Label'] = list(map(labeling_penguin_name, dt3['Species']))

In [871]:
dt3.drop('Species', axis = 1, inplace = True)
dt3.head()

,Culmen_Length,Culmen_Depth,Flipper_Length,DeltaN,DeltaC,Species_Label
0,50.0,15.3,220,8.30515,-25.19017,2
1,49.5,19.0,200,9.63074,-24.34684,0
2,45.1,14.4,210,8.51951,-27.01854,2
3,44.5,14.7,214,8.20106,-26.16524,2
4,49.6,16.0,225,8.38324,-26.84272,2


In [872]:
x_train, x_valid, y_train, y_valid = train_test_split(dt3, target, test_size = 0.2, shuffle = True,
                                                      stratify = dt3['Species_Label'], random_state = 21)
## 종의 비율이 train, valid 비슷하도록 세팅함

`-` 위의 split한 데이터프레임을 reset_index 하게 되면 기존의 인덱스번호가 초기화됨을 잊지말자 

`-` 우선 간단하게 회귀 모형을 만들어보자

`-` 변수별 상관계수가 높으므로 전체 변수를 설정하여 모형을 만들고 유의하지 않은 변수 하나씩 제거하자

### 모델 선택

In [873]:
import statsmodels.formula.api as smf

In [874]:
tmp = pd.concat([x_train, y_train], axis = 1)
tmp.head()

,Culmen_Length,Culmen_Depth,Flipper_Length,DeltaN,DeltaC,Species_Label,Body_Mass
54,51.4,19.0,201,9.43782,-24.36202,0,3950
21,49.8,17.3,198,9.32169,-24.41562,0,3675
2,45.1,14.4,210,8.51951,-27.01854,2,4400
107,46.8,16.5,189,9.65061,-24.48153,0,3650
59,51.1,16.3,220,8.40327,-26.76821,2,6000


In [875]:
result = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
                 Culmen_Depth + Flipper_Length +\
                 DeltaN + DeltaC + Species_Label', data = tmp).fit()

In [876]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     65.70
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           4.65e-29
Time:                        22:05:41   Log-Likelihood:                -636.37
No. Observations:                  88   AIC:                             1287.
Df Residuals:                      81   BIC:                             1304.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -3222.4686   2171.762     -1.484      0.142   -7543.593    1098.656
Culmen_Length     47.4310     13.453      3.526      0.001      20.663      74.199
Culmen_Depth     128.5554     30.371      4.233      0.000      68.126     188.985
Flipper_Length    19.2266      6.577      2.923      0.004       6.140      32.314
DeltaN           -24.8697    107.026     -0.232      0.817    -237.817     188.078
DeltaC            49.8597     69.924      0.713      0.478     -89.266     188.986
Species_Label    706.9739    127.022      5.566      0.000     454.240     959.708
==============================================================================
Omnibus:                        3.068   Durbin-Watson:                   1.621
Prob(Omnibus):                  0.216   Jarque-Bera (JB):                2.561
Skew:                           0.412   Prob(JB):                        0.278
Kurtosis:                       3.142   Cond. No.                     1.23e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.23e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` DeltaN 변수가 유의하지 않게 나온다

`-` 이를 제거하고 다시 회귀분석을 실시하자

In [877]:
result2 = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
                 Culmen_Depth + Flipper_Length +\
                 + DeltaC + Species_Label', data = tmp).fit()

In [878]:
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                     79.75
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           5.06e-30
Time:                        22:05:48   Log-Likelihood:                -636.40
No. Observations:                  88   AIC:                             1285.
Df Residuals:                      82   BIC:                             1300.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -3480.9156   1854.558     -1.877      0.064   -7170.221     208.390
Culmen_Length     47.2143     13.343      3.538      0.001      20.670      73.759
Culmen_Depth     126.9649     29.419      4.316      0.000      68.441     185.489
Flipper_Length    19.2232      6.539      2.940      0.004       6.214      32.232
DeltaC            47.2862     68.642      0.689      0.493     -89.264     183.836
Species_Label    717.5230    117.946      6.083      0.000     482.890     952.156
==============================================================================
Omnibus:                        2.990   Durbin-Watson:                   1.622
Prob(Omnibus):                  0.224   Jarque-Bera (JB):                2.451
Skew:                           0.400   Prob(JB):                        0.294
Kurtosis:                       3.170   Cond. No.                     1.06e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.06e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` DeltaC 변수가 유의하지 않게 나온다

`-` 이를 제거하고 다시 회귀분석을 실시하자

In [879]:
result3 = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
                 Culmen_Depth + Flipper_Length +\
                 + Species_Label', data = tmp).fit()

In [880]:
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     100.2
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           5.98e-31
Time:                        22:05:53   Log-Likelihood:                -636.65
No. Observations:                  88   AIC:                             1283.
Df Residuals:                      83   BIC:                             1296.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -4568.4808    970.052     -4.710      0.000   -6497.874   -2639.087
Culmen_Length     49.5620     12.860      3.854      0.000      23.984      75.140
Culmen_Depth     123.3441     28.854      4.275      0.000      65.955     180.733
Flipper_Length    18.5964      6.455      2.881      0.005       5.757      31.436
Species_Label    680.1134    104.369      6.516      0.000     472.527     887.700
==============================================================================
Omnibus:                        4.545   Durbin-Watson:                   1.617
Prob(Omnibus):                  0.103   Jarque-Bera (JB):                3.827
Skew:                           0.476   Prob(JB):                        0.148
Kurtosis:                       3.371   Cond. No.                     5.51e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.51e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` 이 정도면 된것 같다

`-` x_valid와 y_valid를 통해 모형을 평가하자

### 모델 평가

In [885]:
def mass_predict(dt):
    return -4568.4808 + 49.5620*dt['Culmen_Length'] + dt['Culmen_Depth']*123.3441 + dt['Flipper_Length']*18.5964 + dt['Species_Label']*680.1134

In [886]:
predict_y = mass_predict(x_valid)
predict_y

68     3478.04683
34     3959.72355
20     3757.40676
43     4301.91791
15     5309.15370
105    4015.77190
69     5396.61079
23     3840.46855
111    4105.08565
71     4958.40670
63     3908.15347
76     3549.33378
13     4790.38722
110    3881.97926
91     4130.02143
89     4618.58953
24     3692.12354
46     3802.31431
83     4576.88367
3      4790.04287
113    5683.32773
98     5315.38090
45     4766.52506
dtype: float64

In [887]:
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true - pred)))
    return score

In [888]:
# 실제 정답 값을 준비
real_answer = y_valid.copy()

# 정답과 예측 값을 함수에 넣어 결과를 확인
error = RMSE(real_answer, predict_y)

print(f'이 방정식의 평균 에러는 {error:.2f} 입니다')

이 방정식의 평균 에러는 233.96 입니다


`-` 모델 평가를 마쳤다

`-` test파일을 불러와 예측하고 이를 제출하자

### test 데이터 예측

In [889]:
test = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/test.csv')
test.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837


In [890]:
test.isnull().sum()

id                     0
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    6
Delta 15 N (o/oo)      9
Delta 13 C (o/oo)      8
dtype: int64

`-` test 데이터에 NA가 존재하지만 어짜피 사용하지 않는 변수이므로 무시해도 된다

In [891]:
test.rename(columns = {'Clutch Completion':'Full_Clutch', 'Culmen Length (mm)':'Culmen_Length', 
                     'Culmen Depth (mm)':'Culmen_Depth', 'Flipper Length (mm)':'Flipper_Length',
                     'Delta 15 N (o/oo)':'DeltaN', 'Delta 13 C (o/oo)':'DeltaC', 'Body Mass (g)':'Body_Mass'}, inplace = True) ## 열 이름 변경

In [892]:
test['Species_Label'] = list(map(labeling_penguin_name, test['Species']))

In [893]:
test.head()

,id,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC,Species_Label
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440,0
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425,2
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199,1
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984,0
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837,1


In [894]:
# 예측을 진행
predict_test = mass_predict(test)

# 예측 결과를 확인
predict_test

0      4467.21007
1      5899.09070
2      3891.77948
3      3271.14606
4      3482.81346
          ...    
223    4104.54259
224    4957.29048
225    4666.76835
226    4089.82096
227    3938.73771
Length: 228, dtype: float64

### dacon 대회에 제출하기

In [366]:
# 제출용 sample 파일을 불러옴
submission = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/sample_submission.csv')
submission.head()

,id,Body Mass (g)
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [367]:
# 위에서 구한 예측값을 그대로 넣음
submission['Body Mass (g)'] = predict_test

# 데이터가 잘 들어갔는지 확인
submission

,id,Body Mass (g)
0,0,4467.21007
1,1,5899.09070
2,2,3891.77948
3,3,3271.14606
4,4,3482.81346
...,...,...
223,223,4104.54259
224,224,4957.29048
225,225,4666.76835
226,226,4089.82096


In [370]:
# submission을 csv 파일로 저장
submission.to_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/submission.csv', index = False)

`-` 점수는 329.2286192974

## 회귀 분석 (원핫인코딩)

`-` 라벨 인코딩과 원핫인코딩 모델간의 성능차이를 확인해보자

### 전처리

`-` Species 변수를 수치형으로 변경해주기 위해 원핫인코딩 해주자

In [726]:
dt4 = dt2.copy()
dt4.head()

,Species,Culmen_Length,Culmen_Depth,Flipper_Length,DeltaN,DeltaC
0,Gentoo penguin (Pygoscelis papua),50.0,15.3,220,8.30515,-25.19017
1,Chinstrap penguin (Pygoscelis antarctica),49.5,19.0,200,9.63074,-24.34684
2,Gentoo penguin (Pygoscelis papua),45.1,14.4,210,8.51951,-27.01854
3,Gentoo penguin (Pygoscelis papua),44.5,14.7,214,8.20106,-26.16524
4,Gentoo penguin (Pygoscelis papua),49.6,16.0,225,8.38324,-26.84272


In [727]:
dummy = pd.get_dummies(dt4['Species'])
dummy.head()

,Adelie Penguin (Pygoscelis adeliae),Chinstrap penguin (Pygoscelis antarctica),Gentoo penguin (Pygoscelis papua)
0,0,0,1
1,0,1,0
2,0,0,1
3,0,0,1
4,0,0,1


In [728]:
dt4 = pd.concat([dt4, dummy], axis = 1)

In [729]:
dt4.rename(columns = {'Adelie Penguin (Pygoscelis adeliae)':'Adelie',
                          'Chinstrap penguin (Pygoscelis antarctica)':'Chinstrap',
                          'Gentoo penguin (Pygoscelis papua)':'Gentoo'}, inplace = True)

In [730]:
dt4_Species = dt4['Species'] ## train, vaild 데이터 split을 위한 기준

In [731]:
dt4.head()

,Species,Culmen_Length,Culmen_Depth,Flipper_Length,DeltaN,DeltaC,Adelie,Chinstrap,Gentoo
0,Gentoo penguin (Pygoscelis papua),50.0,15.3,220,8.30515,-25.19017,0,0,1
1,Chinstrap penguin (Pygoscelis antarctica),49.5,19.0,200,9.63074,-24.34684,0,1,0
2,Gentoo penguin (Pygoscelis papua),45.1,14.4,210,8.51951,-27.01854,0,0,1
3,Gentoo penguin (Pygoscelis papua),44.5,14.7,214,8.20106,-26.16524,0,0,1
4,Gentoo penguin (Pygoscelis papua),49.6,16.0,225,8.38324,-26.84272,0,0,1


In [732]:
x_train, x_valid, y_train, y_valid = train_test_split(dt4, target, test_size = 0.2, shuffle = True,
                                                      stratify = dt4['Species'], random_state = 21)
## 종의 비율이 train, valid 비슷하도록 세팅함

In [733]:
x_train.drop('Species', axis = 1, inplace = True)
x_valid.drop('Species', axis = 1, inplace = True)

`-` 정규화를 시켜주자(최대값 $1$, 최소값 $-1$)

`-` 표준화는 회귀보단 분류에 적합하고 정규화는 분류보단 회귀에 적합하다고 한다

`-` 둘다 이상점에 민감하다

In [686]:
# from sklearn.preprocessing import MinMaxScaler

# ## 스케일러 생성
# minmax_scaler = MinMaxScaler()
# minmax_scaler.fit(x_train)

# ## 트레인집합 정규화
# x_train_scaled = minmax_scaler.transform(x_train)
# x_train_scaled_df = pd.DataFrame(data = x_train_scaled, columns = x_train.columns)

# ## 테스트집합 정규화
# x_valid_scaled = minmax_scaler.transform(x_valid)
# x_valid_scaled_df = pd.DataFrame(data = x_valid_scaled, columns = x_valid.columns)

In [687]:
# x_train_scaled_df['Body_Mass'] = list(y_train)

In [688]:
# x_train_scaled_df.head()

`-` 그런데 생각해보니 test 데이터셋은 스케일링을 못함

`-` 그런데 회귀모형은 스케일링된 계수에 의해 추정됨

`-` 예측하려면 스케일링을 하면 안되겠음

`-` 어짜피 회귀모형은 스케일링 하든 안하든 똑같다

In [742]:
tmp2 = pd.concat([x_train, y_train], axis = 1) ## 인덱스 번호를 기준으로 concat

In [743]:
tmp2.head()

,Culmen_Length,Culmen_Depth,Flipper_Length,DeltaN,DeltaC,Adelie,Chinstrap,Gentoo,Body_Mass
65,35.5,16.2,195,8.56192,-25.95541,1,0,0,3350
53,46.6,17.8,193,8.95063,-24.59897,0,1,0,3800
50,45.8,18.9,197,8.18658,-26.45978,1,0,0,4150
110,39.2,18.6,190,9.11006,-25.79549,1,0,0,4250
100,47.4,14.6,212,8.19101,-26.24369,0,0,1,4725


### 모델 선택

In [744]:
rslt = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
               Culmen_Depth + Flipper_Length +\
               Adelie + Chinstrap + Gentoo +\
               DeltaN + DeltaC', data = tmp2).fit()

In [745]:
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     62.23
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           9.62e-30
Time:                        20:46:48   Log-Likelihood:                -631.77
No. Observations:                  88   AIC:                             1280.
Df Residuals:                      80   BIC:                             1299.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -2442.1635   1659.887     -1.471      0.145   -5745.444     861.117
Culmen_Length     41.7500     14.405      2.898      0.005      13.082      70.418
Culmen_Depth     167.3214     40.024      4.181      0.000      87.672     246.971
Flipper_Length    17.2052      7.049      2.441      0.017       3.178      31.232
Adelie          -890.1198    569.535     -1.563      0.122   -2023.530     243.290
Chinstrap      -1554.9619    532.623     -2.919      0.005   -2614.916    -495.008
Gentoo             2.9182    611.074      0.005      0.996   -1213.158    1218.994
DeltaN            -3.2148    105.409     -0.030      0.976    -212.984     206.555
DeltaC            28.2993     70.644      0.401      0.690    -112.286     168.885
==============================================================================
Omnibus:                        7.414   Durbin-Watson:                   1.905
Prob(Omnibus):                  0.025   Jarque-Bera (JB):                6.845
Skew:                           0.627   Prob(JB):                       0.0326
Kurtosis:                       3.541   Cond. No.                     6.67e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.76e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

`-` DeltaN 변수가 가장 유의하지 않으니 제거하자 (Species 관련 변수는 예외)

In [746]:
rslt2 = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
               Culmen_Depth + Flipper_Length +\
               Adelie + Chinstrap + Gentoo +\
               DeltaC', data = tmp2).fit()

In [747]:
rslt2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     73.50
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           1.07e-30
Time:                        20:46:58   Log-Likelihood:                -631.77
No. Observations:                  88   AIC:                             1278.
Df Residuals:                      81   BIC:                             1295.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -2467.7864   1422.709     -1.735      0.087   -5298.531     362.958
Culmen_Length     41.7182     14.279      2.922      0.005      13.308      70.128
Culmen_Depth     167.1613     39.432      4.239      0.000      88.703     245.619
Flipper_Length    17.2079      7.004      2.457      0.016       3.271      31.145
Adelie          -898.7366    491.456     -1.829      0.071   -1876.580      79.107
Chinstrap      -1564.7367    422.767     -3.701      0.000   -2405.910    -723.564
Gentoo            -4.3132    559.713     -0.008      0.994   -1117.966    1109.340
DeltaC            27.9302     69.168      0.404      0.687    -109.693     165.554
==============================================================================
Omnibus:                        7.411   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.025   Jarque-Bera (JB):                6.842
Skew:                           0.626   Prob(JB):                       0.0327
Kurtosis:                       3.546   Cond. No.                     9.29e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.51e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

`-` 유의하지 않은 변수를 제거하니 조정된 결정계수가 올라간 것을 확인할 수 있다

`-` 이번에는 DeltaC 변수가 가장 유의하지 않으니 제거하자

In [754]:
rslt3 = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
               Culmen_Depth + Flipper_Length +\
               Adelie + Chinstrap + Gentoo', data = tmp2).fit()

In [755]:
rslt3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                     89.08
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           1.16e-31
Time:                        20:50:15   Log-Likelihood:                -631.86
No. Observations:                  88   AIC:                             1276.
Df Residuals:                      82   BIC:                             1291.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -2943.8328    792.326     -3.715      0.000   -4520.022   -1367.644
Culmen_Length     42.0173     14.187      2.962      0.004      13.796      70.239
Culmen_Depth     169.0688     38.948      4.341      0.000      91.589     246.549
Flipper_Length    16.5883      6.799      2.440      0.017       3.062      30.114
Adelie         -1072.8467    234.618     -4.573      0.000   -1539.577    -606.117
Chinstrap      -1701.4698    251.827     -6.757      0.000   -2202.434   -1200.506
Gentoo          -169.5163    380.012     -0.446      0.657    -925.480     586.448
==============================================================================
Omnibus:                        8.965   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.011   Jarque-Bera (JB):                8.682
Skew:                           0.678   Prob(JB):                       0.0130
Kurtosis:                       3.726   Cond. No.                     6.45e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.23e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

`-` 이 정도면 된 것 같다

`-` 테스트셋을 통해 모델을 평가하자

### 모델 평가

In [794]:
pred_df = pd.DataFrame(rslt3.params).reset_index().rename(columns = {'index':'var_name', 0:'coefficient'})
pred_df

,var_name,coefficient
0,Intercept,-2943.832806
1,Culmen_Length,42.017287
2,Culmen_Depth,169.068775
3,Flipper_Length,16.588323
4,Adelie,-1072.846737
5,Chinstrap,-1701.469763
6,Gentoo,-169.516306


In [759]:
def mass_predict2(df):
    y_predict = np.array(list(pred_df.query('var_name == "Intercept"')['coefficient']) * len(df))
    for col in df.columns:
        if col in pred_df.set_index('var_name').T.columns:
            y_predict += (np.array(df[col] * pred_df.query('var_name == @col')['coefficient'].to_numpy()))
    return y_predict

In [761]:
predict_y = mass_predict2(x_valid)
predict_y

array([3879.28406818, 3464.06203859, 3600.76003746, 3957.17036717,
       4555.15447372, 4368.86511034, 4607.6304749 , 4367.69701828,
       4269.19787971, 5250.58288746, 4122.72840559, 4317.55133228,
       4791.63216738, 4060.94552841, 3910.03918426, 4764.61042854,
       3814.2468456 , 3547.79085381, 5267.86883389, 4762.77587997,
       5391.49299941, 5260.12115268, 4955.36228951])

In [762]:
# 실제 정답 값을 준비
real_answer = y_valid.copy()

# 정답과 예측 값을 함수에 넣어 결과를 확인
error = RMSE(real_answer, predict_y)

print(f'이 방정식의 평균 에러는 {error:.2f} 입니다')

이 방정식의 평균 에러는 333.44 입니다


`-` 모델 평가를 마쳤다

`-` test파일을 불러와 예측하고 이를 제출하자

### test 데이터 예측

In [763]:
test = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/test.csv')
test.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837


In [764]:
test.isnull().sum()

id                     0
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    6
Delta 15 N (o/oo)      9
Delta 13 C (o/oo)      8
dtype: int64

`-` test 데이터에 NA가 존재하지만 어짜피 사용하지 않는 변수이므로 무시해도 된다

In [716]:
test.rename(columns = {'Clutch Completion':'Full_Clutch', 'Culmen Length (mm)':'Culmen_Length', 
                     'Culmen Depth (mm)':'Culmen_Depth', 'Flipper Length (mm)':'Flipper_Length',
                     'Delta 15 N (o/oo)':'DeltaN', 'Delta 13 C (o/oo)':'DeltaC', 'Body Mass (g)':'Body_Mass'}, inplace = True) ## 열 이름 변경

In [717]:
test.head()

,id,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837


`-` 테스트 데이터셋도 Species에 대해 원핫인코딩을 하자

In [718]:
test2 = test.copy()
test2.head()

,id,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837


In [719]:
dummy = pd.get_dummies(test2['Species'])
dummy.head()

,Adelie Penguin (Pygoscelis adeliae),Chinstrap penguin (Pygoscelis antarctica),Gentoo penguin (Pygoscelis papua)
0,0,1,0
1,0,0,1
2,1,0,0
3,0,1,0
4,1,0,0


In [720]:
test3 = pd.concat([test2, dummy], axis = 1)

In [721]:
test3.rename(columns = {'Adelie Penguin (Pygoscelis adeliae)':'Adelie',
                          'Chinstrap penguin (Pygoscelis antarctica)':'Chinstrap',
                          'Gentoo penguin (Pygoscelis papua)':'Gentoo'}, inplace = True)

In [724]:
test3.head()

,id,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC,Adelie,Chinstrap,Gentoo
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440,0,1,0
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425,0,0,1
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199,1,0,0
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984,0,1,0
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837,1,0,0


In [776]:
# 예측을 진행
predict_test = mass_predict2(test3)

# 예측 결과를 확인
predict_test[:6]

array([4522.86772177, 5891.72393308, 3948.06716922, 3228.79006523,
       3573.64632139, 4184.46973098])

### dacon 대회에 제출하기

In [774]:
# 제출용 sample 파일을 불러옴
submission2 = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/sample_submission.csv')
submission2.head()

,id,Body Mass (g)
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [777]:
# 위에서 구한 예측값을 그대로 넣음
submission2['Body Mass (g)'] = predict_test

# 데이터가 잘 들어갔는지 확인
submission2

,id,Body Mass (g)
0,0,4522.867722
1,1,5891.723933
2,2,3948.067169
3,3,3228.790065
4,4,3573.646321
...,...,...
223,223,4158.047770
224,224,4942.338586
225,225,4606.430920
226,226,4124.571236


In [778]:
# submission을 csv 파일로 저장
submission2.to_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/submission2.csv', index = False)

`-` 점수는 341.1752753861

## 회귀 분석 (라벨 인코딩) + 뒤로부터 제거하는 방법

### 전처리

In [896]:
from sklearn.model_selection import train_test_split

In [897]:
dt = df.iloc[:, :-1]      ## target 제외한 데이터프레임
target = df['Body_Mass']  ## target

`-` 위에서는 범주형 변수를 Species만 사용했는데 이번에는 모든 변수를 사용하자

In [898]:
dt2 = dt.copy() 

In [899]:
dt2.head()

,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC
0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.30515,-25.19017
1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.63074,-24.34684
2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.51951,-27.01854
3,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,44.5,14.7,214,FEMALE,8.20106,-26.16524
4,Gentoo penguin (Pygoscelis papua),Biscoe,No,49.6,16.0,225,MALE,8.38324,-26.84272


`-` 범주형 변수에 라벨인코딩을 해주자

In [900]:
from sklearn.preprocessing import LabelEncoder

dt3 = dt2.copy()
for col in dt2.columns:
    if dt2.dtypes[col] == 'O':
        encoder = LabelEncoder()
        encoder.fit(dt2[col])
        labels = encoder.transform(dt2[col])
        dt3[col] = list(labels) 

In [901]:
dt3.head()  ## 라벨 인코딩이 잘 되었다

,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC
0,2,0,1,50.0,15.3,220,1,8.30515,-25.19017
1,1,1,0,49.5,19.0,200,1,9.63074,-24.34684
2,2,0,1,45.1,14.4,210,0,8.51951,-27.01854
3,2,0,1,44.5,14.7,214,0,8.20106,-26.16524
4,2,0,0,49.6,16.0,225,1,8.38324,-26.84272


In [902]:
x_train, x_valid, y_train, y_valid = train_test_split(dt3, target, test_size = 0.2, shuffle = True,
                                                      stratify = dt3['Species'], random_state = 21)
## 종의 비율이 train, valid 비슷하도록 세팅함

`-` 위의 split한 데이터프레임을 reset_index 하게 되면 기존의 인덱스번호가 초기화됨을 잊지말자 

`-` 우선 간단하게 회귀 모형을 만들어보자

`-` 변수별 상관계수가 높으므로 전체 변수를 설정하여 모형을 만들고 유의하지 않은 변수 하나씩 제거하자

### 모델 선택

In [322]:
import statsmodels.formula.api as smf

In [905]:
tmp3 = pd.concat([x_train, y_train], axis = 1)
tmp3.head()

,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC,Body_Mass
65,0,0,1,35.5,16.2,195,0,8.56192,-25.95541,3350
53,1,1,1,46.6,17.8,193,0,8.95063,-24.59897,3800
50,0,2,1,45.8,18.9,197,1,8.18658,-26.45978,4150
110,0,1,1,39.2,18.6,190,1,9.11006,-25.79549,4250
100,2,0,1,47.4,14.6,212,0,8.19101,-26.24369,4725


In [907]:
rst = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
                 Culmen_Depth + Flipper_Length +\
                 DeltaN + DeltaC + Species +\
                 Island + Full_Clutch + Sex', data = tmp3).fit()

In [908]:
rst.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                     49.77
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           1.06e-28
Time:                        22:10:44   Log-Likelihood:                -629.78
No. Observations:                  88   AIC:                             1280.
Df Residuals:                      78   BIC:                             1304.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2399.8389   2591.857      0.926      0.357   -2760.152    7559.829
Culmen_Length    -10.9524     14.424     -0.759      0.450     -39.668      17.764
Culmen_Depth      13.3529     42.150      0.317      0.752     -70.562      97.268
Flipper_Length    22.2540      6.611      3.366      0.001       9.093      35.415
DeltaN          -321.7528    100.706     -3.195      0.002    -522.244    -121.262
DeltaC             2.6339     68.271      0.039      0.969    -133.284     138.552
Species          200.8958    141.123      1.424      0.159     -80.058     481.849
Island          -136.8211     81.952     -1.670      0.099    -299.976      26.333
Full_Clutch      122.3698    125.092      0.978      0.331    -126.668     371.408
Sex              633.0867    107.041      5.914      0.000     419.984     846.189
==============================================================================
Omnibus:                        1.053   Durbin-Watson:                   1.884
Prob(Omnibus):                  0.591   Jarque-Bera (JB):                0.695
Skew:                           0.210   Prob(JB):                        0.706
Kurtosis:                       3.115   Cond. No.                     1.55e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.55e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` DeltaC 변수가 가장 유의하지 않게 나온다

`-` 이를 제거하고 다시 회귀분석을 실시하자

In [909]:
rst2 = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
                 Culmen_Depth + Flipper_Length +\
                 DeltaN + Species +\
                 Island + Full_Clutch + Sex', data = tmp3).fit()

In [910]:
rst2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     56.71
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           1.36e-29
Time:                        22:11:21   Log-Likelihood:                -629.78
No. Observations:                  88   AIC:                             1278.
Df Residuals:                      79   BIC:                             1300.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2320.9440   1582.322      1.467      0.146    -828.589    5470.477
Culmen_Length    -10.7993     13.779     -0.784      0.436     -38.225      16.627
Culmen_Depth      13.8435     39.931      0.347      0.730     -65.637      93.324
Flipper_Length    22.1730      6.228      3.560      0.001       9.776      34.570
DeltaN          -320.3516     93.334     -3.432      0.001    -506.128    -134.575
Species          202.0020    137.303      1.471      0.145     -71.293     475.297
Island          -136.7533     81.414     -1.680      0.097    -298.804      25.297
Full_Clutch      121.9879    123.909      0.984      0.328    -124.646     368.622
Sex              631.8021    101.085      6.250      0.000     430.597     833.008
==============================================================================
Omnibus:                        1.107   Durbin-Watson:                   1.885
Prob(Omnibus):                  0.575   Jarque-Bera (JB):                0.733
Skew:                           0.215   Prob(JB):                        0.693
Kurtosis:                       3.125   Cond. No.                     9.49e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.49e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` Culmen_Depth 변수가 가장 유의하지 않게 나온다

`-` 이를 제거하고 다시 회귀분석을 실시하자

In [911]:
rst3 = smf.ols(formula = 'Body_Mass ~ Culmen_Length +\
                 Flipper_Length +\
                 DeltaN + Species +\
                 Island + Full_Clutch + Sex', data = tmp3).fit()

In [912]:
rst3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.851
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     65.51
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           1.71e-30
Time:                        22:12:24   Log-Likelihood:                -629.85
No. Observations:                  88   AIC:                             1276.
Df Residuals:                      80   BIC:                             1296.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2412.5143   1551.519      1.555      0.124    -675.107    5500.135
Culmen_Length     -9.7136     13.344     -0.728      0.469     -36.270      16.842
Flipper_Length    22.3154      6.180      3.611      0.001      10.016      34.615
DeltaN          -310.0279     87.968     -3.524      0.001    -485.090    -134.965
Species          178.7892    119.210      1.500      0.138     -58.446     416.025
Island          -133.8125     80.524     -1.662      0.100    -294.061      26.436
Full_Clutch      114.2052    121.186      0.942      0.349    -126.963     355.373
Sex              643.5805     94.680      6.797      0.000     455.161     832.000
==============================================================================
Omnibus:                        0.824   Durbin-Watson:                   1.902
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.505
Skew:                           0.179   Prob(JB):                        0.777
Kurtosis:                       3.101   Cond. No.                     9.33e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.33e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` Culmen_Length 변수가 가장 유의하지 않게 나온다

`-` 이를 제거하고 다시 회귀분석을 실시하자

In [913]:
rst4 = smf.ols(formula = 'Body_Mass ~ Flipper_Length +\
                 DeltaN + Species +\
                 Island + Full_Clutch + Sex', data = tmp3).fit()

In [914]:
rst4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     76.79
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           2.42e-31
Time:                        22:13:39   Log-Likelihood:                -630.14
No. Observations:                  88   AIC:                             1274.
Df Residuals:                      81   BIC:                             1292.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2452.6237   1546.034      1.586      0.117    -623.499    5528.746
Flipper_Length    21.5860      6.081      3.550      0.001       9.487      33.685
DeltaN          -339.1882     78.090     -4.344      0.000    -494.563    -183.813
Species          127.4058     95.785      1.330      0.187     -63.176     317.987
Island          -153.5986     75.578     -2.032      0.045    -303.975      -3.222
Full_Clutch      121.2939    120.443      1.007      0.317    -118.350     360.938
Sex              618.4623     87.912      7.035      0.000     443.545     793.380
==============================================================================
Omnibus:                        1.149   Durbin-Watson:                   1.862
Prob(Omnibus):                  0.563   Jarque-Bera (JB):                0.712
Skew:                           0.200   Prob(JB):                        0.700
Kurtosis:                       3.183   Cond. No.                     9.10e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.1e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` Full_Clutch 변수가 가장 유의하지 않게 나온다

`-` 이를 제거하고 다시 회귀분석을 실시하자

In [915]:
rst5 = smf.ols(formula = 'Body_Mass ~ Flipper_Length +\
                 DeltaN + Species +\
                 Island + Sex', data = tmp3).fit()

In [916]:
rst5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     91.93
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           3.94e-32
Time:                        22:15:05   Log-Likelihood:                -630.68
No. Observations:                  88   AIC:                             1273.
Df Residuals:                      82   BIC:                             1288.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2613.8872   1537.851      1.700      0.093    -445.389    5673.164
Flipper_Length    22.1353      6.057      3.655      0.000      10.086      34.184
DeltaN          -355.4184     76.416     -4.651      0.000    -507.433    -203.404
Species          113.1672     94.744      1.194      0.236     -75.308     301.642
Island          -161.2879     75.198     -2.145      0.035    -310.880     -11.696
Sex              613.6689     87.791      6.990      0.000     439.025     788.312
==============================================================================
Omnibus:                        0.659   Durbin-Watson:                   1.916
Prob(Omnibus):                  0.719   Jarque-Bera (JB):                0.406
Skew:                           0.163   Prob(JB):                        0.816
Kurtosis:                       3.061   Cond. No.                     9.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` Species 변수가 가장 유의하지 않게 나온다

`-` 이를 제거하고 다시 회귀분석을 실시하자

In [917]:
rst6 = smf.ols(formula = 'Body_Mass ~ Flipper_Length +\
                 DeltaN +\
                 Island + Sex', data = tmp3).fit()

In [918]:
rst6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     114.0
Date:                Mon, 27 Dec 2021   Prob (F-statistic):           6.96e-33
Time:                        22:15:48   Log-Likelihood:                -631.44
No. Observations:                  88   AIC:                             1273.
Df Residuals:                      83   BIC:                             1285.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       1490.7290   1220.008      1.222      0.225    -935.818    3917.276
Flipper_Length    27.8102      3.767      7.384      0.000      20.319      35.302
DeltaN          -340.2355     75.544     -4.504      0.000    -490.490    -189.981
Island          -187.2928     72.162     -2.595      0.011    -330.820     -43.766
Sex              567.9853     79.224      7.169      0.000     410.411     725.559
==============================================================================
Omnibus:                        1.038   Durbin-Watson:                   1.836
Prob(Omnibus):                  0.595   Jarque-Bera (JB):                0.836
Skew:                           0.239   Prob(JB):                        0.658
Kurtosis:                       2.989   Cond. No.                     7.16e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.16e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

`-` 유의하지 않은 변수가 없다

`-` 이 정도면 된 것 같다

### 모델 평가

In [919]:
pred_df = pd.DataFrame(rst6.params).reset_index().rename(columns = {'index':'var_name', 0:'coefficient'})

def mass_predict3(df):
    y_predict = np.array(list(pred_df.query('var_name == "Intercept"')['coefficient']) * len(df))
    for col in df.columns:
        if col in pred_df.set_index('var_name').T.columns:
            y_predict += (np.array(df[col] * pred_df.query('var_name == @col')['coefficient'].to_numpy()))
    return y_predict

In [920]:
predict_y = mass_predict3(x_valid)
predict_y

array([4088.99698853, 3262.38680126, 3416.02375236, 4239.24693211,
       4538.74535672, 4153.91328752, 4567.10251469, 4161.50367854,
       4019.94138735, 5308.39312007, 4476.54027674, 3987.33083183,
       4651.82199682, 4250.19204558, 3614.00847867, 4666.17267874,
       4039.99966844, 3785.36130516, 5453.29512891, 4553.3903797 ,
       5498.12713446, 5423.4890208 , 4612.74806186])

In [921]:
# 실제 정답 값을 준비
real_answer = y_valid.copy()

# 정답과 예측 값을 함수에 넣어 결과를 확인
error = RMSE(real_answer, predict_y)

print(f'이 방정식의 평균 에러는 {error:.2f} 입니다')

이 방정식의 평균 에러는 297.96 입니다


`-` 모델 평가를 마쳤다

`-` test파일을 불러와 예측하고 이를 제출하자

### test 데이터 예측

In [922]:
test = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/test.csv')
test.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837


In [923]:
test.isnull().sum()

id                     0
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    6
Delta 15 N (o/oo)      9
Delta 13 C (o/oo)      8
dtype: int64

`-` test 데이터에 NA가 존재하니 성별은 그냥 수컷으로 넣고 나머지는 train 데이터셋의 평균으로 넣자

In [931]:
test['Sex'].fillna('MALE', inplace = True)
test['Delta 15 N (o/oo)'].fillna(dt['DeltaN'].mean(), inplace = True)
test['Delta 13 C (o/oo)'].fillna(dt['DeltaC'].mean(), inplace = True)

In [932]:
test.isnull().sum()

id                     0
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    0
Delta 15 N (o/oo)      0
Delta 13 C (o/oo)      0
dtype: int64

In [933]:
test.rename(columns = {'Clutch Completion':'Full_Clutch', 'Culmen Length (mm)':'Culmen_Length', 
                     'Culmen Depth (mm)':'Culmen_Depth', 'Flipper Length (mm)':'Flipper_Length',
                     'Delta 15 N (o/oo)':'DeltaN', 'Delta 13 C (o/oo)':'DeltaC', 'Body Mass (g)':'Body_Mass'}, inplace = True) ## 열 이름 변경

`-` test 셋도 범주형변수에 대해 라벨 인코딩을 해주자

In [934]:
test3 = test.copy()
for col in test.columns:
    if test.dtypes[col] == 'O':
        encoder = LabelEncoder()
        encoder.fit(test[col])
        labels = encoder.transform(test[col])
        test3[col] = list(labels) 

In [941]:
test3.head()

,id,Species,Island,Full_Clutch,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,DeltaN,DeltaC
0,0,1,1,1,52.0,20.7,210.0,1,9.43146,-24.68440
1,1,2,0,1,55.9,17.0,228.0,1,8.31180,-26.35425
2,2,0,1,1,38.9,18.8,190.0,0,8.36936,-26.11199
3,3,1,1,1,45.2,16.6,191.0,0,9.62357,-24.78984
4,4,0,0,0,37.9,18.6,172.0,0,8.38404,-25.19837


`-` 라벨 인코딩이 잘 되었다

In [952]:
# 예측을 진행
predict_test = mass_predict3(test3)

# 예측 결과를 확인
predict_test[:6]

array([4502.64783971, 5571.47254239, 3739.82251622, 3340.90590658,
       3421.53684881, 4285.72881611])

### dacon 대회에 제출하기

In [953]:
# 제출용 sample 파일을 불러옴
submission3 = pd.read_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/sample_submission.csv')
submission3.head()

,id,Body Mass (g)
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [955]:
# 위에서 구한 예측값을 그대로 넣음
submission3['Body Mass (g)'] = predict_test

# 데이터가 잘 들어갔는지 확인
submission3.head()

,id,Body Mass (g)
0,0,4502.647840
1,1,5571.472542
2,2,3739.822516
3,3,3340.905907
4,4,3421.536849


In [956]:
# submission을 csv 파일로 저장
submission3.to_csv('C:/Users/Jaesu/Dacon-Basic/뗑컨-몸무게-예측/Data/submission3.csv', index = False)

`-` 점수는 336.5351567246